In [1]:
from dataset import ViSFD
from metrics import AspectF1Score, PolarityF1Score
from vnsabsa_model.tokenization_vnsabsa import VnSmartphoneAbsaTokenizer
from modules import LSTM_CNN, SmartphoneBERT
from loss import Loss
from torch.optim import AdamW
from utils import train
from torchinfo import summary
import torch

torch.random.manual_seed(42)

In [2]:
# Train config
BATCH_SIZE = 64
EPOCHS = 40
NUM_GRAD_ACCUMULATE = 2
DEVICE = "cuda"
METRICS = {
    "Aspect F1": AspectF1Score(aspect_thresholds=0.5),
    "Polarity F1": PolarityF1Score()
}

# Datasets
train_set = ViSFD("data", "train")
val_set = ViSFD("data", "dev")
test_set = ViSFD("data", "test")
# Aspect label weights
a_label_weights = len(train_set) / train_set.label_frequencies.size(0) / train_set.label_frequencies

# Modules
tokenizer = VnSmartphoneAbsaTokenizer(
    vocab_file="./pretrained_tokenizer/vn-smartphone-absa-vocab.json", 
    merge_file="./pretrained_tokenizer/vn-smartphone-absa-merges.txt"
)
# model = LSTM_CNN(tokenizer.vocab_size, lstm_hidden_size=768)
model = SmartphoneBERT(tokenizer.vocab_size, num_encoders=4)
loss = Loss(
    a_label_weight=a_label_weights.cuda(),
    aspect_weight=1,
    polarity_weight=1
)
optimizer = AdamW(model.parameters(), 1e-4)

summary(model)

Layer (type:depth-idx)                                            Param #
SmartphoneBERT                                                    --
├─Embedding: 1-1                                                  4,048,896
├─TransformerEncoder: 1-2                                         --
│    └─ModuleList: 2-1                                            --
│    │    └─TransformerEncoderLayer: 3-1                          3,546,624
│    │    └─TransformerEncoderLayer: 3-2                          3,546,624
│    │    └─TransformerEncoderLayer: 3-3                          3,546,624
│    │    └─TransformerEncoderLayer: 3-4                          3,546,624
│    └─LayerNorm: 2-2                                             1,536
├─AspectClassifier: 1-3                                           --
│    └─Sequential: 2-3                                            --
│    │    └─Dropout: 3-5                                          --
│    │    └─Linear: 3-6                                     

In [ ]:
train(
    tokenizer=tokenizer,
    model=model,
    train_set=train_set,
    val_set=val_set,
    batch_size=BATCH_SIZE,
    num_grad_accumulate=NUM_GRAD_ACCUMULATE,
    epochs=EPOCHS,
    optimizer=optimizer,
    loss_fn=loss,
    metrics=METRICS,
    device=DEVICE,
    enable_record_loss=True
)

In [ ]:
# states = torch.load("pretrained_model/smartphone-bert.pt")
states = torch.load("pretrained_model/smartphone-lstm-cnn.pt")
model.load_state_dict(states)

<All keys matched successfully>

In [ ]:
from utils import absa_eval

absa_eval(
    tokenizer=tokenizer,
    model=model,
    test_set=test_set,
    reduction="mean", # Or "none" for aspect-wise evaluation
    batch_size=BATCH_SIZE
)

Epoch 1 - Polarity F1 Score Evaluation: 100%|██████████| 35/35 [00:04<00:00,  8.65it/s]


{'Aspect F1': 0.72042, 'Polarity F1': 0.6795499920845032}